In [1]:
from data_utils import splitter

splitter(3,4,seed = 5)

(array([6, 2, 4]), array([1, 0, 5, 3]))

In [5]:
dic = {"training_losses" : [22]}

assert "training_losses" in dic.keys() and len(dic["training_losses"]) == 0

AssertionError: 

In [6]:
assert len(dic["hhh"]) == 0

KeyError: 'hhh'